In [1]:

import pandas as pd


## Initial Data Exploration

The first step was to explore the raw datasets to understand their structure,
data types, and potential quality issues before applying any transformations.

In [3]:
orders = pd.read_csv("../data/raw_orders.csv")
customers = pd.read_csv("../data/raw_customers.csv")
products = pd.read_csv("../data/raw_products.csv")


In [4]:
orders.head(), customers.head(), products.head()


(   order_id  customer_id  order_date  product_id  quantity  unit_price
 0      1001          1.0  2024-01-05         101       1.0       19.99
 1      1002          2.0  05/01/2024         102       2.0       29.99
 2      1002          2.0  05/01/2024         102       2.0       29.99
 3      1003          3.0  2024/01/07         103       NaN       15.00
 4      1004          NaN  2024-01-08         104       1.0       49.99,
    customer_id customer_name  country signup_date
 0            1         Alice  Ireland  2023-11-01
 1            2           Bob  Ireland  2023/11/15
 2            3       Charlie    Spain  15-12-2023
 3            4           NaN  Ireland  2023-12-20
 4            5           Eva      NaN  2024-01-01,
    product_id product_name   category  cost
 0         101      T-Shirt    Apparel   8.5
 1         102        Shoes   Footwear  18.0
 2         103          Hat    Apparel   5.0
 3         104       Jacket  Outerwear  30.0
 4         105        Socks    appa

In [5]:
orders.info(), customers.info(), products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     7 non-null      int64  
 1   customer_id  6 non-null      float64
 2   order_date   7 non-null      object 
 3   product_id   7 non-null      int64  
 4   quantity     6 non-null      float64
 5   unit_price   7 non-null      float64
dtypes: float64(3), int64(2), object(1)
memory usage: 468.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    5 non-null      int64 
 1   customer_name  4 non-null      object
 2   country        4 non-null      object
 3   signup_date    5 non-null      object
dtypes: int64(1), object(3)
memory usage: 292.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (to

(None, None, None)

## Step 2 – Data Quality Assessment

Based on the initial inspection using `info()` and `head()`, the following data quality issues were identified:

### Orders table
- One order record is missing a `customer_id`, which prevents proper linkage to the customers table.
- One order has a missing `quantity` value.
- The `order_date` column is stored as text and contains multiple date formats.
- `unit_price` is correctly stored as a numeric type.
- Duplicate order IDs

### Customers table
- One customer record is missing a `customer_name`.
- One customer record is missing a `country`.
- The `signup_date` column is stored as text and uses inconsistent date formats.

### Products table
- The `category` column is stored as text and may contain inconsistent casing.
- The `cost` column is already stored as a numeric type.

These issues were addressed in the following cleaning steps to ensure data consistency and reliability for analysis.


In [6]:
orders = orders.drop_duplicates(subset=['order_id'])


In [7]:
orders = orders.dropna(subset=["customer_id"])
orders["customer_id"] = orders["customer_id"].astype(int)


In [8]:
orders["quantity"] = orders["quantity"].fillna(1).astype(int)


In [9]:
orders["order_date"] = pd.to_datetime(orders["order_date"], errors='coerce')


In [ ]:
orders["order_date"].isna().sum()


np.int64(3)

In [11]:
customers["customer_name"] = customers["customer_name"].fillna("Unknown")
customers["country"] = customers["country"].fillna("Unknown")

In [12]:
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")


In [13]:
customers["signup_date"].isna().sum()


np.int64(2)

In [14]:
products["category"] = products["category"].str.lower()


In [15]:
orders.info()
customers.info()
products.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     5 non-null      int64         
 1   customer_id  5 non-null      int64         
 2   order_date   2 non-null      datetime64[ns]
 3   product_id   5 non-null      int64         
 4   quantity     5 non-null      int64         
 5   unit_price   5 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 280.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    5 non-null      int64         
 1   customer_name  5 non-null      object        
 2   country        5 non-null      object        
 3   signup_date    3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(

## Step 3 – Data Cleaning Decisions

Data cleaning actions were applied based on business impact and analytical relevance.
Critical issues affecting joins and revenue calculations were resolved, while non-critical
missing attributes were handled through reasonable default values.

All date fields were standardized, numeric fields validated, and categorical values normalized
to ensure consistency before loading the data into a relational database.


In [17]:
orders = orders.dropna(subset=["order_date"])


In [19]:
orders.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     2 non-null      int64         
 1   customer_id  2 non-null      int64         
 2   order_date   2 non-null      datetime64[ns]
 3   product_id   2 non-null      int64         
 4   quantity     2 non-null      int64         
 5   unit_price   2 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 112.0 bytes


In [20]:
products["category"] = (
    products["category"]
    .str.strip()
    .str.lower()
)

In [21]:
orders[orders["unit_price"] <= 0]
products[products["cost"] <= 0]


,product_id,product_name,category,cost


In [22]:
orders.info()
customers.info()
products.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 6
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     2 non-null      int64         
 1   customer_id  2 non-null      int64         
 2   order_date   2 non-null      datetime64[ns]
 3   product_id   2 non-null      int64         
 4   quantity     2 non-null      int64         
 5   unit_price   2 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 112.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   customer_id    5 non-null      int64         
 1   customer_name  5 non-null      object        
 2   country        5 non-null      object        
 3   signup_date    3 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(

In [23]:
orders["revenue"] = orders["quantity"] * orders["unit_price"]


In [24]:
orders = orders.merge(
    products[["product_id", "cost"]],
    on="product_id",
    how="left"
)


In [25]:
orders["total_cost"] = orders["quantity"] * orders["cost"]
orders["profit"] = orders["revenue"] - orders["total_cost"]


In [28]:
orders[["revenue", "total_cost", "profit"]].describe()


,revenue,total_cost,profit
count,2.00,2.0,2.00
mean,19.99,8.5,11.49
std,0.00,0.0,0.00
min,19.99,8.5,11.49
25%,19.99,8.5,11.49
50%,19.99,8.5,11.49
75%,19.99,8.5,11.49
max,19.99,8.5,11.49


In [29]:
orders["margin_pct"] = orders["profit"] / orders["revenue"]


In [32]:
orders["product_id"].value_counts()


product_id
101    2
Name: count, dtype: int64

In [34]:
orders[["product_id", "revenue", "profit", "margin_pct"]]


,product_id,revenue,profit,margin_pct
0,101,19.99,11.49,0.574787
1,101,19.99,11.49,0.574787


In [35]:
orders.groupby("product_id")[["revenue", "profit", "margin_pct"]] \
      .mean() \
      .sort_values("margin_pct", ascending=False)


,revenue,profit,margin_pct
product_id,,,
101,19.99,11.49,0.574787


In [36]:
product_performance = (
    orders.groupby("product_id")
    .agg(
        total_revenue=("revenue", "sum"),
        total_profit=("profit", "sum"),
        avg_margin_pct=("margin_pct", "mean")
    )
    .sort_values("total_profit", ascending=False)
)

product_performance


,total_revenue,total_profit,avg_margin_pct
product_id,,,
101,39.98,22.98,0.574787
